In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "training_linear_models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Linear Regression Training


In Machine Learning, vectors are represented as column vectors which are 2D arrays with a single column. If the models' parameter vector(containing bias term and feature weights) and instance's feature vector are column vectors, then prediction  is a matrix multiplication. This gives the same prediction but it will be represented as a single cell matrix rather than a scalar-value.

The Normal Equation
The objective of a regression is to find the best fit whilst keeping the cost function low. Hence our objective is to find the value of features that minimizes cost function. We will first introduce the concept of a closed form solution (AKA a mathematical equation that gives this result directly - Normal Equation)